## Lego DataSet Exploratory Data Analysis
### DataSci200 Project 2

In [9]:
# Import Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display

pd.set_option('display.max_rows', 1000)

In [10]:
# Import Individual Lego DataFrames
sets = pd.read_csv("data/sets.csv.gz")
themes = pd.read_csv("data/themes.csv.gz")
# Import IMDB data
imdb_titles = pd.read_csv("C:/Users/mhurth/OneDrive - PureWest Energy/Desktop/MIDS/200/title.basics.tsv.gz", sep= "\t")

C:\Users\mhurth\AppData\Local\Temp\1\ipykernel_9544\603823613.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb_titles = pd.read_csv("C:/Users/mhurth/OneDrive - PureWest Energy/Desktop/MIDS/200/title.basics.tsv.gz", sep= "\t")


In [11]:
# Clean Up the IMDB dataset
imdb_titles.replace('\\N', np.nan, inplace=True)
imdb_titles.dropna(subset=["startYear","titleType","primaryTitle", "runtimeMinutes"], inplace=True)
# Filter start year for imdb data
imdb_titles["startYear"] = imdb_titles["startYear"].astype(int)
imdb_titles = imdb_titles[(imdb_titles["startYear"] >= 1950) & (imdb_titles["startYear"] <= 2022)]
# Filter to only Movies 
imdb_titles = imdb_titles.loc[imdb_titles["titleType"] == "movie"]
# Clean Up column type to filter out short films
imdb_titles["runtimeMinutes"] = imdb_titles["runtimeMinutes"].astype(int)
imdb_titles = imdb_titles[imdb_titles["runtimeMinutes"]>=60]

In [12]:
# Filter to years for sets
sets = sets.loc[sets.year <= 2022]
# Merging Sets and Themes to look at most common themes over time
sets_and_themes_df = sets.merge(themes, left_on="theme_id", right_on="id", how='left', suffixes=('_set', "_theme"))
sets_and_themes_df = sets_and_themes_df.loc[sets_and_themes_df.year <= 2022]
# Cleaning Sets and Themes by populating parent theme, and handiling parent theme nulls
sets_and_themes_df.parent_id = sets_and_themes_df.parent_id.fillna(sets_and_themes_df.id)
id_themes = sets_and_themes_df[["id","name_theme"]].drop_duplicates()
sets_and_themes_df = sets_and_themes_df.merge(id_themes, left_on="parent_id", right_on="id", how="left", suffixes=("","_parent"))
sets_and_themes_df.drop(columns=["id_parent"],inplace=True)
sets_and_themes_df.rename(columns={"name_theme_parent":"parent_theme"}, inplace=True)
sets_and_themes_df.loc[sets_and_themes_df.parent_id == 147, "parent_theme"] = "Pirates I, Pirates II, Pirates III"
sets_and_themes_df.loc[sets_and_themes_df.parent_id == 296, "parent_theme"] = "Dino Island, Desert, Orient Expedition, Jungle"
sets_and_themes_df.loc[sets_and_themes_df.parent_id == 561, "parent_theme"] = "The Lord of the Rings, The Hobbit"

#### Movie Related Themes
- Star Wars
- Harry Potter
- Jurassic World
- Batman
- The Hobbit/The Lord of the Rings
- Indiana Jones
- Pirates Of The Caribean
- Minions
- Trolls

In [21]:
franchise_dict = {"Star Wars": {"regex": r'\bstar wars\b', 
                               "movies": ["Star Wars: Episode IV - A New Hope", 
                                          "Star Wars: Episode V - The Empire Strikes Back", 
                                          "Star Wars: Episode VI - Return of the Jedi", 
                                          "Star Wars: Episode I - The Phantom Menace",
                                          "Star Wars: Episode II - Attack of the Clones",
                                          "Star Wars: Episode III - Revenge of the Sith",
                                          "Star Wars: Episode VII - The Force Awakens",
                                          "Star Wars: Episode VIII - The Last Jedi",
                                          "Star Wars: Episode IX - The Rise of Skywalker",
                                          "Solo: A Star Wars Story",
                                          "Rogue One: A Star Wars Story",
                                          "Solo: A Star Wars Story"]},
                  "Harry Potter": {"regex": r'\bharry potter\b', 
                                  "movies":["Harry Potter and the Sorcerer's Stone",
                                            "Harry Potter and the Chamber of Secrets",
                                            "Harry Potter and the Prisoner of Azkaban",
                                            "Harry Potter and the Goblet of Fire",
                                            "Harry Potter and the Order of the Phoenix",
                                            "Harry Potter and the Half-Blood Prince",
                                            "Harry Potter and the Deathly Hallows: Part 1",
                                            "Harry Potter and the Deathly Hallows: Part 2"]},
                 "Jurassic Park": {"regex": r'\bJurassic\b', 
                                  "movies": ["Jurassic Park", 
                                             "The Lost World: Jurassic Park", 
                                             "Jurassic Park III", 
                                             "Jurassic World", 
                                             "Jurassic World: Fallen Kingdom", 
                                             "Jurassic World: Dominion"]},
                  "Batman": {"regex": r'\bbatman\b', 
                                  "movies": ["Batman: The Movie",
                                             "Batman",
                                             "Batman Returns",
                                             "Batman: Mask of the Phantasm",
                                             "Batman Forever",
                                             "Batman & Robin",
                                             "Batman Begins",
                                             "The Dark Knight",
                                             "The Dark Knight Rises",
                                             "Batman v Superman: Dawn of Justice",
                                             "The Lego Batman Movie",
                                             "The Batman"]},
                 "Lord of the Rings": {"regex": r'\bLord of the Rings\b', 
                                       "movies": ["The Hobbit: An Unexpected Journey",
                                                  "The Hobbit: The Desolation of Smaug",
                                                  "The Hobbit: The Battle of the Five Armies",
                                                  "The Lord of the Rings: The Fellowship of the Ring",
                                                  "The Lord of the Rings: The Two Towers",
                                                  "The Lord of the Rings: The Return of the King"]},
                  "Indiana Jones": {"regex": r'\bIndiana Jones\b', 
                                    "movies": ["Raiders of the Lost Ark",
                                               "Indiana Jones and the Temple of Doom",
                                               "Indiana Jones and the Last Crusade",
                                               "Indiana Jones and the Kingdom of the Crystal Skull"]},
                 "Pirates of the Caribbean": {"regex": r'\bPirates of the Caribbean\b',
                                              "movies": ["Pirates of the Caribbean: The Curse of the Black Pearl",
                                                         "Pirates of the Caribbean: Dead Man's Chest",
                                                         "Pirates of the Caribbean: At World's End",
                                                         "Pirates of the Caribbean: On Stranger Tides",
                                                         "Pirates of the Caribbean: Dead Men Tell No Tales"]},
                 "Minions": {"regex": r'\bMinions\b',
                             "movies": ["Minions",
                                        "Minions: The Rise of Gru"]},
                 "Trolls": {"regex": r'\bTrolls\b',
                            "movies": ["Trolls",
                                       "Trolls 2: World Tour"]}}


In [22]:
def null_years_to_zero(df, df_imdb):
    min_year = np.min(df_imdb["startYear"])
    max_year = np.max(df_imdb["startYear"])
    df = df.set_index('year')
    df_new = pd.DataFrame({"year":range(min_year-5, min(2022,max_year+5)+1), "set_num": 0}).set_index("year")
    df_new["set_num"]=df["set_num"]
    df_new.fillna(0, inplace=True)
    df_new.reset_index(inplace=True)
    return df_new

In [23]:
def plot_movies_and_lego_sets(lego_df, imdb_df, movie_dict, name):
    pattern = movie_dict[name]["regex"]
    movie_list = movie_dict[name]["movies"]
    lego = lego_df.loc[lego_df["parent_theme"].str.contains(pattern, regex=True, case=False)]
    lego = pd.DataFrame(lego.groupby("year")["set_num"].count()).reset_index()
    imdb = imdb_df[imdb_df["primaryTitle"].isin(movie_list)].sort_values(by="startYear").reset_index()
    cmap = plt.cm.get_cmap('hsv', len(imdb))
    for i in range(len(imdb)):
        year = imdb.loc[i, "startYear"]
        title = imdb.loc[i, "primaryTitle"]
        color = cmap(i)
        plt.axvline(x=year, c=color, label=f"{year}: {title}")
    lego = null_years_to_zero(lego, imdb)
    sns.lineplot(data=lego, x="year", y="set_num", marker="o")
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.ylabel("Number of Sets")
    plt.xlabel("Year")
    plt.title(f"{name} Lego Set and Movie Releases by Year")
    plt.show()

In [26]:
movie_dropdown = widgets.Dropdown(options=list(franchise_dict.keys()), description = "Theme")
def update_plot(name):
    plot_movies_and_lego_sets(sets_and_themes_df, imdb_titles, franchise_dict, name)
widgets.interactive(update_plot, name=movie_dropdown)  

interactive(children=(Dropdown(description='Theme', options=('Star Wars', 'Harry Potter', 'Jurassic Park', 'Ba…

#### Observations
- StarWars
- Harry Potter
- Jurassic Park
- Indiana Jones
- Batman
- LOTR
- Pirates of the Carribean
- Minions
- Trolls